In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tools import add_constant
from statsmodels.iolib.summary2 import summary_col, summary_params
from scipy.stats import t # t-распределение
import seaborn as sns

In [ ]:
# подключим датасет loanapp по ссылке
loanapp_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/econometrica/main/econometrica2/data-csv/loanapp.csv', na_values=(' ', '', '  '))


In [ ]:
loanapp_df.head(n=5)

,occ,loanamt,action,msa,suffolk,appinc,typur,unit,married,dep,...,approve,mortno,mortperf,mortlat1,mortlat2,chist,multi,loanprc,thick,white
0,1,89,1,1120,0,72,0,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.754237,0.0,1
1,1,128,3,1120,0,74,0,1.0,1.0,1.0,...,0,0,1,0,0,1,0.0,0.800000,1.0,1
2,1,128,1,1120,0,84,3,1.0,0.0,0.0,...,1,0,1,0,0,1,0.0,0.895105,1.0,1
3,1,66,1,1120,0,36,0,1.0,1.0,0.0,...,1,0,1,0,0,0,0.0,0.600000,0.0,1
4,1,120,1,1120,0,59,8,1.0,1.0,0.0,...,1,0,1,0,0,1,0.0,0.895522,0.0,1


In [ ]:
# размеры датасета loanapp
loanapp_df.shape


(1989, 59)

In [ ]:
# информация о датасете loanapp
loanapp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 59 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   occ       1989 non-null   int64  
 1   loanamt   1989 non-null   int64  
 2   action    1989 non-null   int64  
 3   msa       1989 non-null   int64  
 4   suffolk   1989 non-null   int64  
 5   appinc    1989 non-null   int64  
 6   typur     1989 non-null   int64  
 7   unit      1985 non-null   float64
 8   married   1986 non-null   float64
 9   dep       1986 non-null   float64
 10  emp       1989 non-null   int64  
 11  yjob      1989 non-null   int64  
 12  self      1989 non-null   int64  
 13  atotinc   1989 non-null   int64  
 14  cototinc  1989 non-null   float64
 15  hexp      1989 non-null   float64
 16  price     1989 non-null   float64
 17  other     1989 non-null   float64
 18  liq       1989 non-null   float64
 19  rep       1980 non-null   float64
 20  gdlin     1989 non-null   int6

In [ ]:
# число пропусков по каждой переменной
loanapp_df.isna().sum()

occ           0
loanamt       0
action        0
msa           0
suffolk       0
appinc        0
typur         0
unit          4
married       3
dep           3
emp           0
yjob          0
self          0
atotinc       0
cototinc      0
hexp          0
price         0
other         0
liq           0
rep           9
gdlin         0
lines         0
mortg         0
cons          0
pubrec        0
hrat          0
obrat         0
fixadj        0
term          0
apr           0
prop          0
inss          0
inson         0
gift          0
cosign        0
unver         0
review        0
netw          0
unem          0
min30       183
bd            0
mi            0
old           0
vr            0
sch           0
black         0
hispan        0
male         15
reject        0
approve       0
mortno        0
mortperf      0
mortlat1      0
mortlat2      0
chist         0
multi         4
loanprc       0
thick         9
white         0
dtype: int64

In [ ]:
#зададим спецификацию модели через формулу
mod_lpm = smf.ols(formula='approve~appinc+appinc^2+mortno+unem+dep+male+married+yjob+self', data=loanapp_df)
# подгонка модели
res_lpm_hc = mod_lpm.fit(cov_type='HC3')
print(res_lpm_hc.summary(slim=True))


                            OLS Regression Results                            
Dep. Variable:                approve   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.017
No. Observations:                1971   F-statistic:                     4.689
Covariance Type:                  HC3   Prob (F-statistic):           3.53e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8735      0.025     35.652      0.000       0.826       0.922
appinc         0.0020      0.004      0.537      0.592      -0.005       0.009
appinc ^ 2    -0.0021      0.004     -0.573      0.567      -0.009       0.005
mortno         0.0758      0.015      4.990      0.000       0.046       0.106
unem          -0.0068      0.004     -1.693      0.091      -0.015       0.001
dep           -0.0178      0.008     -2.349      0.0

In [ ]:
# Коэфициенты модели с округлением до 3-х десятичных знаков
res_lpm_hc.params.round(3)

Intercept     0.874
appinc        0.002
appinc ^ 2   -0.002
mortno        0.076
unem         -0.007
dep          -0.018
male          0.003
married       0.047
yjob         -0.001
self         -0.031
dtype: float64

In [ ]:
res_lpm_ols = mod_lpm.fit(cov_type='nonrobust')
# Сравнение моделей
print(summary_col([res_lpm_hc, res_lpm_ols], model_names=['Robust', 'Non-robust'], stars=True))


                 Robust  Non-robust
-----------------------------------
Intercept      0.8735*** 0.8735*** 
               (0.0245)  (0.0227)  
appinc         0.0020    0.0020    
               (0.0037)  (0.0037)  
appinc ^ 2     -0.0021   -0.0021   
               (0.0037)  (0.0037)  
mortno         0.0758*** 0.0758*** 
               (0.0152)  (0.0161)  
unem           -0.0068*  -0.0068** 
               (0.0040)  (0.0035)  
dep            -0.0178** -0.0178** 
               (0.0076)  (0.0072)  
male           0.0034    0.0034    
               (0.0214)  (0.0203)  
married        0.0470**  0.0470*** 
               (0.0188)  (0.0177)  
yjob           -0.0006   -0.0006   
               (0.0062)  (0.0067)  
self           -0.0312   -0.0312   
               (0.0252)  (0.0225)  
R-squared      0.0216    0.0216    
R-squared Adj. 0.0172    0.0172    
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [ ]:
# Коэфициенты модели с округлением до 3-х десятичных знаков
res_lpm_ols.params.round(3)

Intercept     0.874
appinc        0.002
appinc ^ 2   -0.002
mortno        0.076
unem         -0.007
dep          -0.018
male          0.003
married       0.047
yjob         -0.001
self         -0.031
dtype: float64

2 t-тест
Значимость коэффициентов
Тестируем гипотезу  H0:β=0

Тестовая статистика
t=β^s.e.(β)

Критическое  tcr=tdf=n−k−1(α)

2.1 approve equation #1

In [ ]:
# робастные t-статистики для каждого коэффициента с округлением до 3-х десятичных знаков
np.round(res_lpm_hc.tvalues, 3)

Intercept     35.652
appinc         0.537
appinc ^ 2    -0.573
mortno         4.990
unem          -1.693
dep           -2.349
male           0.157
married        2.507
yjob          -0.104
self          -1.241
dtype: float64

In [ ]:
# Число наблюдений в модели, число регрессоров и степени свободы для t_cr
res_lpm_hc.nobs, res_lpm_hc.df_model, res_lpm_hc.df_resid

(1971.0, 9.0, 1961.0)

In [ ]:
# 1%-критическое значение t-распределения
np.round(t.ppf(q=1-0.01/2, df=res_lpm_hc.df_resid), 3)

2.578

In [ ]:
# Результаты t-теста для коэффициентов (неробастные s.e.)
summary_params(res_lpm_ols, alpha=0.01)

,Coef.,Std.Err.,t,P>|t|,[0.005,0.995]
Intercept,0.873540,0.022698,38.484581,7.276304e-242,0.815015,0.932064
appinc,0.001985,0.003685,0.538485,5.903031e-01,-0.007518,0.011487
appinc ^ 2,-0.002121,0.003688,-0.575073,5.653080e-01,-0.011631,0.007389
mortno,0.075783,0.016070,4.715931,2.575715e-06,0.034350,0.117216
unem,-0.006808,0.003471,-1.961404,4.997326e-02,-0.015758,0.002141
dep,-0.017826,0.007199,-2.476189,1.336315e-02,-0.036387,0.000735
male,0.003350,0.020329,0.164799,8.691189e-01,-0.049064,0.055765
married,0.047028,0.017653,2.663955,7.786019e-03,0.001511,0.092545
yjob,-0.000641,0.006684,-0.095854,9.236464e-01,-0.017875,0.016594
self,-0.031220,0.022480,-1.388783,1.650566e-01,-0.089182,0.026741


In [ ]:
# проверим значимость коэффициентов
df_ols = np.round(summary_params(res_lpm_ols, alpha=0.01), 3)
df_ols['significance'] = df_ols.apply(lambda x: 'Значим' if x['P>|t|']<0.01 else 'Незначим', axis=1)
df_ols

,Coef.,Std.Err.,t,P>|t|,[0.005,0.995],significance
Intercept,0.874,0.023,38.485,0.000,0.815,0.932,Значим
appinc,0.002,0.004,0.538,0.590,-0.008,0.011,Незначим
appinc ^ 2,-0.002,0.004,-0.575,0.565,-0.012,0.007,Незначим
mortno,0.076,0.016,4.716,0.000,0.034,0.117,Значим
unem,-0.007,0.003,-1.961,0.050,-0.016,0.002,Незначим
dep,-0.018,0.007,-2.476,0.013,-0.036,0.001,Незначим
male,0.003,0.020,0.165,0.869,-0.049,0.056,Незначим
married,0.047,0.018,2.664,0.008,0.002,0.093,Значим
yjob,-0.001,0.007,-0.096,0.924,-0.018,0.017,Незначим
self,-0.031,0.022,-1.389,0.165,-0.089,0.027,Незначим


In [ ]:
# Результаты t-теста для коэффициентов (робастные s.e.)
summary_params(res_lpm_hc, alpha=0.01)

,Coef.,Std.Err.,t,P>|t|,[0.005,0.995]
Intercept,0.873540,0.024502,35.652021,2.192338e-278,0.810427,0.936652
appinc,0.001985,0.003699,0.536564,5.915691e-01,-0.007542,0.011512
appinc ^ 2,-0.002121,0.003704,-0.572683,5.668596e-01,-0.011661,0.007419
mortno,0.075783,0.015187,4.990083,6.035345e-07,0.036665,0.114902
unem,-0.006808,0.004022,-1.692518,9.054729e-02,-0.017169,0.003553
dep,-0.017826,0.007590,-2.348550,1.884666e-02,-0.037377,0.001725
male,0.003350,0.021393,0.156601,8.755593e-01,-0.051755,0.058455
married,0.047028,0.018756,2.507301,1.216570e-02,-0.001285,0.095342
yjob,-0.000641,0.006164,-0.103940,9.172172e-01,-0.016519,0.015238
self,-0.031220,0.025153,-1.241186,2.145369e-01,-0.096011,0.033571


In [ ]:
# проверим значимость коэффициентов
df_hc = np.round(summary_params(res_lpm_hc, alpha=0.01), 3)
df_hc['significance'] = df_hc.apply(lambda x: 'Значим' if x['P>|t|']<0.01 else 'Незначим', axis=1)
df_hc

,Coef.,Std.Err.,t,P>|t|,[0.005,0.995],significance
Intercept,0.874,0.025,35.652,0.000,0.810,0.937,Значим
appinc,0.002,0.004,0.537,0.592,-0.008,0.012,Незначим
appinc ^ 2,-0.002,0.004,-0.573,0.567,-0.012,0.007,Незначим
mortno,0.076,0.015,4.990,0.000,0.037,0.115,Значим
unem,-0.007,0.004,-1.693,0.091,-0.017,0.004,Незначим
dep,-0.018,0.008,-2.349,0.019,-0.037,0.002,Незначим
male,0.003,0.021,0.157,0.876,-0.052,0.058,Незначим
married,0.047,0.019,2.507,0.012,-0.001,0.095,Незначим
yjob,-0.001,0.006,-0.104,0.917,-0.017,0.015,Незначим
self,-0.031,0.025,-1.241,0.215,-0.096,0.034,Незначим


In [ ]:
res_lpm_hc.t_test('mortno=0, married=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0758      0.015      4.990      0.000       0.046       0.106
c1             0.0470      0.019      2.507      0.012       0.010       0.084